# Generici e gerarchia

Se `S` $\prec$ `T` non è vero che ≠ `G<S>` $\prec$ `G<T>` (dove `G` è un qualche tipo generico).

Si considerino ad esempio

In [1]:
List<Integer> li = new ArrayList<>();
List<Object> lo;

Se `li` fosse un sottotipo di `lo` l'assegnamento seguente sarebbe possibile (senza *cast*) 

In [2]:
lo = (List)li; 

Una volta ottenuto l'*alias* `lo` potremmo usarlo per aggiungere di tutto a `ls`

In [3]:
lo.add(new Object());

true

ma ovviamente, una volta estratto da `ls` (come `String`) un oggetto qualunque potrebbe dare adito ad un errore di conversione

In [4]:
li.get(0);

EvalException: class java.lang.Object cannot be cast to class java.lang.Integer (java.lang.Object and java.lang.Integer are in module java.base of loader 'bootstrap')

Abbiamo già visto che il punto, in questo caso, è poter aggiungere e togliere elementi del tipo parametrico (avendo sia la garnzia della *type safety* che evitando l'uso del *cast*).

In [5]:
li.add(Integer.valueOf(1));
Integer i = li.get(1);

## Array e gerarchia

Si osservi che la situazione è ben **diversa** per gli array: è infatti vero che se `S` $\prec$ `T`, allora `S[]` $\prec$ `T[]`.

In [6]:
Integer[] ai = new Integer[10];
Object[] ao;

In questo caso, infatti, l'assegmaneto è possibile senza *cast* (per via della relazione di sottotipo)

In [7]:
ao = ai;

Il fatto è che, per gli array, l'assegnamento a `ao` può essere controllato a *run-time* perché l'array conserva l'informazione circa il tipo dei suoi elementi (cosa che non accade per i tipi generici).

In [8]:
ao[0] = new Object();

EvalException: java.lang.Object

L'eccezione `ArrayStoreException` è proprio il modo in cui la VM segnala (a runtime) che è impossibile assegnare un oggetto ad un elemento di `as` (tramite l'*alias* `ao`).

Ovviamente è corretto l'assegnamento

In [9]:
ai[0] = Integer.valueOf(1);

1

che viene eseguito senza errore.
 
Nel caso degli array quindi la *type safety* può essee garantita anche "trasportando" sugli array la relazione di sottotipo dgli elementi.

# Metodi generici

La mancanza di relazioni di tipo tra `G<S>` e `G<T>` anche qualora `S` $\prec$ `T` è particolarmente grave non tanto per il caso degli *alias* visto in precendeza, quanto per il caso dei metodi.

Supponiamo di voler scrivere il metodo `add` in grado di sommare i valori in doppia precisione corrispondenti ad un elenco di numeri

In [10]:
public static double add(List<Number> lst) {
    double sum = 0;
    for (Number n : lst) sum += n.doubleValue();
    return sum;
}

Tutto bene se lo usiamo con una lista del tipo del parametro

In [11]:
List<Number> nums = List.of(1, 2.5, 3);
add(nums)

6.5

Ma per via della mancanza della relazione di sottotipo, non possiamo usarla per sommare una lista di interi

In [12]:
List<Integer> ints = List.of(1, 2, 3);
add(ints);

CompilationException: 

Una possibile soluzione è rendere il metodo **generico** ed indicare un **bound** nella dichiarazione dei parametri di tipo

In [13]:
public static <T extends Number> double add(List<T> lst) {
    double sum = 0;
    for (T n : lst) sum += n.doubleValue();
    return sum;
}

In [14]:
add(ints);

6.0

Consideriamo ora un metodo `copy` in grado di copiare elementi da una lista sorgente ad una destinazione; un primo tentativo di implementazione (attraverso un metodo generico) potrebbe essere

In [15]:
public static <T> void copy(List<T> src, List<T> dst) {
    for (T t : src) dst.add(t);
}

che infatti funziona egregiamente su coppie di liste di interi

In [16]:
List<Integer> dupInts = new ArrayList<>();
copy(ints, dupInts);
dupInts

[1, 2, 3]

Ma che succede se volessimo copiare da una lista di interi ad una di numeri?

In [17]:
List<Number> dupNums = new ArrayList<>();
copy(ints, dupNums);

CompilationException: 

Incorreremmo di nuovo in un problema legato all'assenza di una relazione gerarchica tra i tipi; anche in questo caso, possiamo risolverlo con un *bound*

In [18]:
public static <T, S extends T> void copy(List<S> src, List<T> dst) {
    for (S t : src) dst.add(t);
}

In [19]:
copy(ints, dupNums);
dupNums

[1, 2, 3]

L'uso dei bound sui tipi di parametro dei metodi generici è una prima risposta al problema della mancanza di una gerarchia tra i generici.

Ma è necessaria una soluzione più versatile, in grado di permetterci di dare un tipo (generico) sensato agli argomenti dei metodi.m

# Wildcard

Esiste un parametro di tipo **wildcard** `?` che intuitivamente sta per "qualunque tipo", ragion per cui vale `G<T>` $\prec$`G<?>` (per ogni `T`), ad esempio

In [20]:
List<?> lw;

lw = lo;
lw = li;

Il problema è che in questo modo non è possibile garantire alcuna *type safety*, infatti non c'è verso di scrivere, o leggere, dalla lista

In [21]:
lw.add(Integer.valueOf(1));

CompilationException: 

In [22]:
Integer i = lw.get(1);

CompilationException: 

Tale "libertà" può essere vincolata in vario modo, così che abbia senso usare tipi parametrici basati su wildcard.

## Upper bound

Il caso più semplice è quello degli **upper bound** della forma `? extends T` che consentono di introdurre le seguenti relazione di sottotipo

* per ogni `T`, `G<T>` $\prec$ `G<? extends T>` 
* se `S` $\prec$ `T`, allora `G<? extends S>` $\prec$  `G<? extends T>`.

Ragionando per transitività, se `S` $\prec$ `T`, si ha `G<S>` $\prec$ `G<? extends T>` che, in somma, permette di concludere che il generico basato sull'*upper bound del supertipo* (`G<? extends T>`) è supertipo sia di `G<S>` che di `G<T>` (che pure sono tra loro inconfrontabili dal punto di vista della gerarchia).

### Esempio: consumatore

Un esempio d'uso può chiarire l'obiettivo di tali bound. Immaginiamo di avere un metodo in grado di agire su oggetti di tipo `T`, esso potrà riceverne una lista specificata come `List<T>` ma, certamente, anche come `List<S>` (se `S` $\prec$ `T`); per questa ragione ha senso che il tipo del sua parametro sia `List<? extends T>`.

Ad esempio, consideriamo il metodo `add` visto in precedenza: a questo punto ha senso abbia un parametro di tipo `List<? extends Number>` che è supertipo di `List<Integer>` e `List<Double>`.

In [23]:
static double add(List<? extends Number> lst) {
    double sum = 0;
    for (Number n : lst) sum += n.doubleValue();
    return sum;
}

In [24]:
List<Integer> li = Arrays.asList(1, 2, 3);
List<Double> ld = Arrays.asList(1.5, 2.5, 3.5);

In [25]:
add(li);

6.0

In [26]:
add(ld);

7.5

Il metodo `add` viene chiamato **consumatore** perché legge i valori della lista, se è in grado di gestire il tipo `Number` sarà in grado di gestire i sottotipi. Il consumatore può gestire gli elementi di *al più* un certo tipo, per questa ragione il suo parametro ha un *upper bound*.

## Lower bound

Cosa accadrebbe però se volessimo scrivere un metodo che, invece di leggere i valori dalla lista, volesse aggiungerne? 

Non possiamo segiure l'approccio precedente: volendo aggiungere valori di tipo `T` non possiamo farlo in una lista di tipo `List<? extends T>`, perché sappiamo che se `S` $\prec$ `T` ad essa può corrispondere anche una `List<S>` e finiremmo col mettere oggeti del supertipo in una lista di sottotipi! Seguendo il ragionamento dei bound nei metodi generici vorremmo scrivere una cosa del tipo `T extends ?`, ma questo non è sintatticamente ammesso.

A tal fine vengono invece introdotti i **lower bound** della forma `? super S`che consentono di introdurre le seguenti relazioni di sottotipo

* per ogni `T`, `G<T>` $\prec$ `G<? super T>` 
* se `S` $\prec$ `T`, allora `G<? super T>` $\prec$  `G<? super S>`

si osservi che, nella seconda relazione, l'ordine dei generici è rovesciato rispetto a prima.

Ragionando per transitività, se `S` $\prec$ `T`, si ha `G<T>` $\prec$ `G<? super S>` che, in somma, permette di concludere che: il generico basato sul *lower bound del sottotipo* (`G<? super S>`) è supertipo sia di `G<S>` che di `G<T>`.

### Esempio: produttore

Un esempio d'uso può chiarire meglio il punto. Immaginiamo di avere un metodo che genera elementi di tipo `S`, esso potrà aggiungerli ad una lista specificata come `List<S>` ma, certamente, anche come `List<T>` (se `S` $\prec$ `T`); per questa ragione ha senso che il tipo del suo parametro sia `List<? super S>`.

Ad esempio, il metodo `putInts` che aggiunge degli interi ad una lista, ha senso che abbia un parametro di tipo `List<? super Integer>` che è supertipo di `List<Integer>`, ma certamente andrebbe bene anche aggiungerli ad una lista di tipo `List<Numbner>` — ma si badi bene, se il parametro fosse di tipo `List<Number>`, che non è un supertimpo di `List<Integer>`, il metodo non funzionerebbe come vogliamo.

In [27]:
static void putInts(List<? super Integer> lst) {
    for (int i = 0; i < 10; i++) lst.add(Integer.valueOf(i));
}

In [28]:
List<Integer> li = new ArrayList<>();
List<Number> ln = new ArrayList<>();

In [29]:
putInts(li);
li

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [30]:
putInts(ln);
ln

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Il metodo `putInts` viene chiamato **produttore** perché aggiunge valori alla lista, se gli elementi sono `Integer` potrà darli in pasto ad una lista in grado di accettare elementi *almeno* di quel tipo, per questa ragione il suo parametro ha un *lower bound*.